In [1]:
import warnings
import pandas as pd
import torch
from torch import device, cuda, autocast
from torch.cuda.amp import GradScaler
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import wandb
from random import sample
from scipy.sparse import load_npz

import flyvision
from flyvision_ans import DECODING_CELLS
from flyvision.utils.activity_utils import LayerActivity
from from_image_to_video import image_paths_to_sequences
from from_retina_to_connectome_funcs import from_retina_to_model, get_cell_type_indices, compute_voronoi_averages, from_retina_to_connectome
from logs_to_wandb import log_images_to_wandb, log_running_stats_to_wandb
from from_video_to_training_batched_funcs import get_files_from_directory, select_random_videos, paths_to_labels, \
    load_custom_sequences
from from_retina_to_connectome_utils import hex_to_square_grid, initialize_results_df, predictions_and_corrects_from_model_results, update_results_df, update_running_loss, get_decision_making_neurons
from adult_models import FullAdultModel


warnings.filterwarnings(
    'ignore',
    message='invalid value encountered in cast',
    category=RuntimeWarning,
    module='wandb.sdk.data_types.image'
)

torch.manual_seed(1234)
dtype = torch.float32

device_type = "cuda" if cuda.is_available() else "cpu"
#device_type = "cpu"
DEVICE = device(device_type)

TRAINING_DATA_DIR = "images/easy_v2"
TESTING_DATA_DIR = "images/easy_images"
VALIDATION_DATA_DIR = "images/easyval_images"

debugging = True
debug_length = 20
wandb_ = False
wandb_images_every = 100
small = True
small_length = 2000

num_epochs = 1
batch_size = 1

dropout = .1
max_lr = 0.01
base_lr = 0.001
weight_decay = 0.0001
NUM_CONNECTOME_PASSES=5

use_one_cycle_lr = False

model_config = {
    "debugging": debugging,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "dropout": dropout,
    "base_lr": base_lr,
    "max_lr": max_lr,
    "weight_decay": weight_decay,
    "num_connectome_passes": NUM_CONNECTOME_PASSES,
}

/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)
/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch_geometric/typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home

In [2]:
# init stuff
extent, kernel_size = 15, 13
decision_making_vector = get_decision_making_neurons(dtype).to(DEVICE)
receptors = flyvision.rendering.BoxEye(extent=extent, kernel_size=kernel_size)
network_view = flyvision.NetworkView(flyvision.results_dir / "opticflow/000/0000")
network = network_view.init_network(chkpt="best_chkpt")
classification = pd.read_csv("adult_data/classification_clean.csv")
root_id_to_index = pd.read_csv("adult_data/root_id_to_index.csv")
dt = 1 / 100 # some parameter from flyvision
last_good_frame = 2
cell_type_plot = "TmY18"

cell_type_indices = get_cell_type_indices(classification, root_id_to_index, DECODING_CELLS)

training_videos = get_files_from_directory(TRAINING_DATA_DIR)
test_videos = get_files_from_directory(TESTING_DATA_DIR)
validation_videos = get_files_from_directory(TESTING_DATA_DIR)

if small:
    training_videos = sample(training_videos, small_length)
    test_videos = sample(test_videos, small_length)
    validation_videos = sample(validation_videos, int(small_length / 5))

if len(training_videos) == 0:
    print("I can't find any training images or videos!")

In [3]:
synaptic_matrix = load_npz("adult_data/synaptic_matrix_sparse.npz")

model = FullAdultModel(
    synaptic_matrix,
    decision_making_vector,
    root_id_to_index.shape[0],
    NUM_CONNECTOME_PASSES,
    batch_size=batch_size,
    dtype=dtype,
).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=base_lr)
scaler = GradScaler()

# Initialize the loss function
criterion = BCEWithLogitsLoss()

In [4]:
import scipy

from from_retina_to_connectome_utils import create_csr_input


if wandb_:
    wandb.init(project="adult_connectome", config=model_config)

model.train()

results = initialize_results_df()
probabilities = []
accuracies = []
already_selected = []
running_loss = 0.0
total_correct = 0
total = 0

iterations = debug_length if debugging else len(training_videos) // batch_size

for i in tqdm(range(iterations)):
    batch_files, already_selected = select_random_videos(
        training_videos, batch_size, already_selected
    )
    labels = paths_to_labels(batch_files)
    batch_sequences = image_paths_to_sequences(batch_files)
    rendered_sequences = receptors(batch_sequences)

    layer_activations = []
    for rendered_sequence in rendered_sequences:
        # rendered sequences are in RGB; move it to 0-1 for better training
        rendered_sequence = torch.div(rendered_sequence, 255)
        simulation = network.simulate(rendered_sequence[None], dt)
        layer_activations.append(
            LayerActivity(simulation, network.connectome, keepref=True)
        )

    if wandb_ and i % wandb_images_every == 0:
        la_0 = hex_to_square_grid(
            layer_activations[0][cell_type_plot].squeeze()[-last_good_frame].cpu().numpy()
            ),
        log_images_to_wandb(batch_sequences[0], rendered_sequences[0], la_0, batch_files[0], frame=last_good_frame, cell_type=cell_type_plot)

    voronoi_averages_df = compute_voronoi_averages(
        layer_activations, classification, DECODING_CELLS, last_good_frame=last_good_frame
    )
    # normalize column wise (except last column) using apply
    values_cols = voronoi_averages_df.columns != "index_name"
    voronoi_averages_df.loc[:, values_cols] = voronoi_averages_df.loc[:, values_cols].apply(
        lambda x: (x - x.min()) / (x.max() - x.min()), axis=0
        )

    activation_df = from_retina_to_connectome(
        voronoi_averages_df, classification, root_id_to_index
    )
    del layer_activations, rendered_sequences, rendered_sequence, simulation

    torch.cuda.empty_cache()

    if wandb_:
        wandb.watch(model, criterion, log="all", log_freq=20)

    optimizer.zero_grad()

    # inputs = create_csr_input(activation_df)
    inputs = torch.tensor(activation_df.values, dtype=dtype, device=DEVICE)
    labels = torch.tensor(labels, dtype=dtype, device=DEVICE).unsqueeze(1)

    out = model(inputs)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()

    # Calculate run parameters
    predictions, labels_cpu, correct = predictions_and_corrects_from_model_results(out, labels)
    results = update_results_df(results, batch_files, predictions, labels_cpu, correct)
    running_loss += update_running_loss(loss, inputs)
    total += labels.shape[0]
    total_correct += correct.sum()

    if wandb_:
        log_running_stats_to_wandb(0, i, running_loss, total_correct, total, results)

print(f"Finished training with loss {running_loss / total} and accuracy {total_correct / total}")
torch.cuda.empty_cache()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 67.08 GiB. GPU 0 has a total capacity of 7.58 GiB of which 6.99 GiB is free. Process 129168 has 27.52 MiB memory in use. Including non-PyTorch memory, this process has 478.00 MiB memory in use. Of the allocated memory 268.14 MiB is allocated by PyTorch, and 23.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [58]:
already_selected_validation = []
# Assuming batch_size is defined
for _ in tqdm(range(len(validation_videos) // batch_size)):
    batch_files, already_selected_validation = select_random_videos(validation_videos, batch_size, already_selected_validation)

    labels = paths_to_labels(batch_files)  # Convert paths to labels
    batch_sequences = load_custom_sequences(batch_files)  # Load and preprocess the video sequences
    
    # Assuming receptors is a function that processes your sequences
    rendered_sequences = receptors(batch_sequences)
    
    layer_activations = []
    for rendered_sequence in rendered_sequences:
        simulation = network.simulate(rendered_sequence[None], dt)
        layer_activations.append(LayerActivity(simulation, network.connectome, keepref=True))
        
    del rendered_sequences, simulation
    torch.cuda.empty_cache()
    
    # Preparing the data for the model, similar to training
    inputs, labels = from_retina_to_model(layer_activations, labels, DECODING_CELLS, last_good_frame, classification, root_id_to_index)
    torch.cuda.empty_cache()

    model.eval()  # Set the model to evaluation mode
    val_loss = []
    with torch.no_grad():  # Disable gradient computation
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).unsqueeze(-1).float()
        
        with autocast(device_type):
            predictions = model(inputs)
            # Assuming your criterion and evaluation metrics are defined similarly to training
            loss = criterion(predictions, labels)
            val_loss.append(loss.item())
            # Calculate other metrics if necessary, e.g., accuracy
            
            # Log validation metrics to WandB
            wandb.log({"validation_loss": loss.item()})
            # Log other metrics similarly

  0%|          | 0/1960 [00:00<?, ?it/s]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


When cuda breaks:


In [4]:
!sudo rmmod nvidia_uvm
!sudo modprobe nvidia_uvm

[sudo] password for eudald: 
[sudo] password for eudald: 
